In [22]:
import torch
from model import Seq2SeqSpellchecker, CharacterVocabulary, count_parameters

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Device: cpu


In [23]:
vocab = CharacterVocabulary("data/char_vocab.json")

print(f"Vocabulary size: {len(vocab)}")
print(f"Special tokens: {vocab.special_tokens}")


Vocabulary size: 37
Special tokens: ['<PAD>', '<SOS>', '<EOS>', '<UNK>']


In [24]:
model = Seq2SeqSpellchecker(
    vocab_size=len(vocab),
    embedding_dim=256,
    encoder_dim=512,
    decoder_dim=512,
    attention_dim=256,
    num_layers=3,
    dropout=0.3
).to(device)

print(f"Model initialized with {count_parameters(model):,} parameters")


Model initialized with 25,158,693 parameters


In [25]:
checkpoint = torch.load(
    "checkpoints/best_model_continued.pt",
    map_location=device
)

model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print("Model loaded successfully")
print(f"Epoch: {checkpoint['epoch']}")
print(f"Validation loss: {checkpoint['val_loss']:.4f}")


Model loaded successfully
Epoch: 20
Validation loss: 0.5271


In [26]:
def correct_word(word: str) -> str:
    if not word or not isinstance(word, str):
        return word

    src = torch.tensor([vocab.encode(word)]).to(device)
    src_lengths = torch.tensor([len(word)]).to(device)

    with torch.no_grad():
        prediction = model.predict(
            src,
            src_lengths,
            max_length=30,
            sos_idx=vocab.sos_idx,
            eos_idx=vocab.eos_idx
        )

    return vocab.decode(prediction[0].cpu().tolist())


In [27]:
test_word = "გამარჰონა"
print(test_word, "→", correct_word(test_word))


გამარჰონა → გამარჯობა


In [28]:
test_cases = [
    ("გამარჰონა", "გამარჯობა", "substitution"),
    ("მადლობ", "მადლობა", "missing character"),
    ("კარგა", "კარგად", "missing character"),
    ("ცუდად", "ცუდად", "correct"),
    ("ნხვამდის", "ნახვამდის", "missing character"),

    ("საქართველო", "საქართველო", "correct"),
    ("თბილისი", "თბილისი", "correct"),
    ("ქართული", "ქართული", "correct"),
    ("კომპუტერი", "კომპიუტერი", "missing character"),

    ("პროგამა", "პროგრამა", "missing character"),
    ("ინტენრეტი", "ინტერნეტი", "swap"),
    ("კალენდსრი", "კალენდარი", "substitution"),
    ("დღს", "დღეს", "missing character"),
    ("სწავლა", "სწავლა", "correct"),
    ("მუსიკა", "მუსიკა", "correct"),

    ("აკდემია", "აკადემია", "missing character"),
    ("უნვიერსიტეტი", "უნივერსიტეტი", "swap"),
    ("ბიბლიოთეკა", "ბიბლიოთეკა", "correct"),
    ("ტელეივზორი", "ტელევიზორი", "swap"),
    ("მსწავლე", "მოსწავლე", "correct"),
    ("სკოლა", "სკოლა", "correct"),
    ("მასწვლებელი", "მასწავლებელი", "missing character"),
    ("ცდნა", "ცოდნა", "missing character"),
    ("განათლება", "განათლება", "correct"),
    ("იტორია", "ისტორია", "missing character"),
]


In [29]:
correct = 0

for inp, expected, desc in test_cases:
    out = correct_word(inp)
    ok = out == expected
    correct += ok
    status = "✓" if ok else "✗"
    print(f"{status} {inp:15} → {out:15} ({desc})")

print(f"\nAccuracy: {correct}/{len(test_cases)}")


✓ გამარჰონა       → გამარჯობა       (substitution)
✗ მადლობ          → მადლობ          (missing character)
✗ კარგა           → კარგა           (missing character)
✓ ცუდად           → ცუდად           (correct)
✓ ნხვამდის        → ნახვამდის       (missing character)
✓ საქართველო      → საქართველო      (correct)
✓ თბილისი         → თბილისი         (correct)
✓ ქართული         → ქართული         (correct)
✓ კომპუტერი       → კომპიუტერი      (missing character)
✓ პროგამა         → პროგრამა        (missing character)
✓ ინტენრეტი       → ინტერნეტი       (swap)
✓ კალენდსრი       → კალენდარი       (substitution)
✓ დღს             → დღეს            (missing character)
✓ სწავლა          → სწავლა          (correct)
✓ მუსიკა          → მუსიკა          (correct)
✓ აკდემია         → აკადემია        (missing character)
✓ უნვიერსიტეტი    → უნივერსიტეტი    (swap)
✓ ბიბლიოთეკა      → ბიბლიოთეკა      (correct)
✓ ტელეივზორი      → ტელევიზორი      (swap)
✓ მსწავლე         → მოსწავლე        (correct)
✓ სკოლა  

In [32]:
error_stats = {"correct": 0, "wrong": 0}

for inp, expected, _ in test_cases:
    if correct_word(inp) == expected:
        error_stats["correct"] += 1
    else:
        error_stats["wrong"] += 1

print(error_stats)


{'correct': 23, 'wrong': 2}


In [30]:
def test_custom_word(word):
    corrected = correct_word(word)
    if word == corrected:
        print(f"✓ '{word}' is already correct")
    else:
        print(f"Input : {word}")
        print(f"Output: {corrected}")


In [31]:
test_custom_word("პროგამა")


Input : პროგამა
Output: პროგრამა
